In [1]:
%pip install itables tqdm

import math
import pandas as pd
import random
from tqdm import tqdm
from scipy.stats import entropy
from itertools import permutations
from numpy import NaN

def safe_log2(x):
    return math.log2(x) if x > 0 else 0


def tuple_to_string(num_tuple):
    return ''.join(map(str, num_tuple))


def string_to_tuple(input_string):
    return tuple(map(int, input_string))


def calculate_bulls_cows(source, target):
    if len(source) != len(target):
        raise ValueError("Input arrays must have the same length")

    bulls = sum(s == t for s, t in zip(source, target))
    common_digits = set(source) & set(target)
    cows = sum(min(source.count(digit), target.count(digit)) for digit in common_digits) - bulls

    return bulls, cows


def get_pattern_distribution_dict(candidates):
    pattern_distribution_dict = {}
    bulls_cows_dict = {}
    for i, source in tqdm(enumerate(candidates), total=len(candidates)):
        pattern_dist = {}
        for j, target in enumerate(candidates):
            if i < j:
                bulls_cows = calculate_bulls_cows(source, target)
                bulls_cows_dict[source, target] = bulls_cows
                bulls_cows_dict[target, source] = bulls_cows
            elif i == j:
                bulls_cows_dict[source, source] = (len(source), 0)
            
            guess_result = bulls_cows_dict[source, target]
            if tuple_to_string(guess_result) in pattern_dist:
                pattern_dist[tuple_to_string(guess_result)] += 1
            else:
                pattern_dist[tuple_to_string(guess_result)] = 1

        pattern_distribution_dict[source] = pattern_dist

    return pattern_distribution_dict



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
class BullsNCows:
    def __init__(self, digits=4):
        permute = permutations([i for i in range(10)], digits)
        self.candidates = [tuple_to_string(p) for p in list(permute) if p[0]!=0]

        self.attempts = 0
        self.digits = digits
        self.secret = random.choice(self.candidates)
        self.summary = [{
            "candidate_count": len(self.candidates),
            "candidate_entropy": safe_log2(len(self.candidates)),
        }]


    def calc_entropy_dict(self):
        entropy_dict = {}
        pattern_distribution_dict = get_pattern_distribution_dict(self.candidates)

        for candidate in pattern_distribution_dict:
            pattern_dist = pattern_distribution_dict[candidate]
            entropy_dict[candidate] = entropy(list(pattern_dist.values()), base=2)

        return sorted(entropy_dict.items(), reverse=True)


    def next(self):
        # if False:
        if self.attempts > 0:
            entropy_dict = self.calc_entropy_dict()
            print(entropy_dict)
            guess, guess_entropy = entropy_dict[0]
        else:
            guess = random.choice(self.candidates)
            guess_entropy = NaN

        self.attempts += 1
        guess_result = calculate_bulls_cows(self.secret, guess)
        self.candidates = [c for c in self.candidates if calculate_bulls_cows(guess, c) == guess_result]
        self.summary.append({
            "guess": guess,
            "guess_result": guess_result,
            "guess_entropy": guess_entropy,
            "candidate_count": len(self.candidates),
            "candidate_entropy": safe_log2(len(self.candidates)),
        })

        return self


game = BullsNCows(3)
while len(game.candidates) > 1:
    game.next()

print(f'\nSecret: {game.secret}')
pd.DataFrame.from_dict(game.summary)

100%|██████████| 3/3 [00:00<00:00, 2031.47it/s]


[('653', 0.9182958340544894), ('536', 0.9182958340544894), ('365', 0.9182958340544894)]


100%|██████████| 2/2 [00:00<00:00, 2153.69it/s]

[('536', 1.0), ('365', 1.0)]

Secret: 536


,candidate_count,candidate_entropy,guess,guess_result,guess_entropy
0,648,9.339850,NaN,NaN,NaN
1,3,1.584963,563,"(1, 2)",NaN
2,2,1.000000,653,"(0, 3)",0.918296
3,1,0.000000,536,"(3, 0)",1.000000
